<a href="https://colab.research.google.com/github/malluandcompany/Projects/blob/main/jobbank_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 pandas


In [ ]:
!pip install -q requests beautifulsoup4 pandas

import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

BASE = "https://www.jobbank.gc.ca"
SEARCH_URL = "https://www.jobbank.gc.ca/jobsearch/jobsearch?sort=M&searchstring=data+analyst"
PAGES = 2
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

EMPTY = ""

# ---------------------------------------------------------------------------
# Helper functions
# ---------------------------------------------------------------------------

def unique(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out


def get_posting_links(search_html):
    soup = BeautifulSoup(search_html, "html.parser")
    anchors = soup.select('a[href*="/jobsearch/jobposting/"]')
    links = []
    for a in anchors:
        href = a.get("href", "")
        if "/jobsearch/jobposting/" in href:
            links.append(urljoin(BASE, href))
    return unique(links)


def clean_employer(raw):
    if not raw:
        return EMPTY
    txt = re.sub(r'\bEmployer\b[: ]*', '', raw, flags=re.I)
    txt = re.sub(r'\bEmployer Details\b', '', txt, flags=re.I)
    txt = re.sub(r'\bAuto List\b', '', txt, flags=re.I)
    txt = re.sub(r'\s{2,}', ' ', txt).strip()
    return txt


def split_location(raw):
    """Return City, Province, PostalCode; empty if not found."""
    if not raw or not isinstance(raw, str):
        return EMPTY, EMPTY, EMPTY

    text = raw.strip()
    text = re.sub(r'^(Location|Lieu de travail)\s*[:\-]?\s*', '', text, flags=re.I).strip()

    postal_match = re.search(r'\b([A-Z]\d[A-Z])\s?(\d[A-Z]\d)\b', text)
    postal = f"{postal_match.group(1)} {postal_match.group(2)}" if postal_match else EMPTY

    prov_match = re.search(r'\b(AB|BC|MB|NB|NL|NS|NT|NU|ON|PE|QC|SK|YT)\b', text)
    province = prov_match.group(1) if prov_match else EMPTY

    city = EMPTY
    if province:
        before_prov = text.split(province)[0]
        parts = [p.strip() for p in before_prov.split(',') if p.strip()]
        city = parts[-1] if parts else before_prov.strip()
    else:
        parts = [p.strip() for p in text.split(',') if p.strip()]
        city = parts[0] if parts else EMPTY

    return city, province, postal


DATE_RE = re.compile(
    r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}\b'
)

def extract_date(soup):
    t = soup.select_one('time[datetime]')
    if t:
        txt = t.get_text(" ", strip=True)
        m = DATE_RE.search(txt)
        if m:
            return m.group(0)
        return t.get("datetime", EMPTY)
    txt = soup.get_text(" ", strip=True)
    m = DATE_RE.search(txt)
    return m.group(0) if m else EMPTY


def extract_noc(soup):
    a = soup.select_one('a[href*="noc.esdc.gc.ca"]')
    if a:
        m = re.search(r'\b(\d{4,5})\b', a.get_text(" ", strip=True))
        if m:
            return m.group(1)
    txt = soup.get_text(" ", strip=True)
    m = re.search(r'\b(\d{4,5})\b', txt)
    return m.group(1) if m else EMPTY


def extract_employment_type(soup):
    text = soup.get_text(" ", strip=True).lower()
    keywords = ["full time", "part time", "permanent", "temporary", "contract", "casual", "seasonal"]
    found = [k.title() for k in keywords if k in text]
    return ", ".join(found) if found else EMPTY


def extract_work_arrangement(soup):
    text = soup.get_text(" ", strip=True).lower()
    if any(k in text for k in ["remote", "work from home", "telework"]):
        return "Remote/Hybrid"
    if any(k in text for k in ["on site", "on-site"]):
        return "On-site"
    return EMPTY


def extract_source_site_and_url(soup):
    for a in soup.select('a[href]'):
        href = a["href"]
        if "jobbank.gc.ca" not in href:
            site = urlparse(href).netloc.replace("www.", "")
            return site, href
    return EMPTY, EMPTY


def parse_posting(url):
    r = requests.get(url, headers=headers, timeout=20)
    if r.status_code != 200:
        return {"URL": url, "Status": r.status_code}

    soup = BeautifulSoup(r.text, "html.parser")

    title = soup.select_one("h1").get_text(strip=True) if soup.select_one("h1") else EMPTY

    employer_el = soup.find(string=re.compile("Employer", re.I))
    employer = clean_employer(employer_el.strip()) if employer_el else EMPTY

    location_el = soup.find(string=re.compile("Location", re.I))
    location_raw = location_el.parent.get_text(strip=True) if location_el else EMPTY
    city, province, postal = split_location(location_raw)

    wage_el = soup.find(string=re.compile("Wage", re.I)) or soup.find(string=re.compile("Salary", re.I))
    wage = wage_el.parent.get_text(strip=True) if wage_el else EMPTY

    date_posted = extract_date(soup)
    noc = extract_noc(soup)
    emp_type = extract_employment_type(soup)
    work_arr = extract_work_arrangement(soup)
    site, link = extract_source_site_and_url(soup)

    return {
        "Title": title,
        "Employer": employer,
        "City": city,
        "Province": province,
        "PostalCode": postal,
        "Wage": wage,
        "DatePosted": date_posted,
        "NOC": noc,
        "EmploymentType": emp_type,
        "WorkArrangement": work_arr,
        "SourceSite": site,
        "SourceURL": link,
        "URL": url
    }

# ---------------------------------------------------------------------------
# Crawl JobBank
# ---------------------------------------------------------------------------

all_rows = []
for p in range(1, PAGES + 1):
    page_url = f"{SEARCH_URL}&page={p}"
    print("Fetching:", page_url)
    r = requests.get(page_url, headers=headers, timeout=20)
    if r.status_code != 200:
        print("Failed:", page_url)
        continue
    links = get_posting_links(r.text)
    print(f"Page {p} → {len(links)} job links")

    for u in links:
        all_rows.append(parse_posting(u))
        time.sleep(1.25)

df = pd.DataFrame(all_rows).drop_duplicates(subset=["URL"]).reset_index(drop=True)
df.to_csv("jobbank_jobs.csv", index=False)
print("Saved", len(df), "jobs → jobbank_jobs.csv")
df.head(10)


Fetching: https://www.jobbank.gc.ca/jobsearch/jobsearch?sort=M&searchstring=data+analyst&page=1
Page 1 → 25 job links
Fetching: https://www.jobbank.gc.ca/jobsearch/jobsearch?sort=M&searchstring=data+analyst&page=2
Page 2 → 25 job links
Saved 50 jobs → jobbank_jobs.csv


,Title,Employer,City,Province,PostalCode,Wage,DatePosted,NOC,EmploymentType,WorkArrangement,SourceSite,SourceURL,URL
0,big data analyst,Employers,,,,Median wage,"October 03, 2025",21211,Full Time,On-site,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
1,big data analyst,Employers,,,,Median wage,"October 18, 2025",21211,"Full Time, Casual",On-site,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
2,data quality analystLMIA requested,Employers,,,,Median wage,"September 04, 2025",21223,"Full Time, Permanent, Temporary",Remote/Hybrid,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
3,data mining analyst,Employers,,,,Median wage,"September 30, 2025",21211,,On-site,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
4,data mining analystStudent job,Employers,,,,Median wage,"September 24, 2025",21211,Full Time,On-site,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
5,"analyst, database",Employers,,,,Median wage,"October 16, 2025",21223,Full Time,On-site,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
6,data mining analyst,Employers,,,,Median wage,"October 03, 2025",21211,,On-site,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
7,data analyst - informatics and systems,Employers,,,,Median wage,"October 20, 2025",21223,"Full Time, Permanent, Temporary",Remote/Hybrid,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
8,data analyst - informatics and systems,Employers,,,,Median wage,"October 11, 2025",21223,Part Time,On-site,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...
9,data analyst - informatics and systems,Employers,,,,Median wage,"October 10, 2025",21223,,On-site,,#searchString,https://www.jobbank.gc.ca/jobsearch/jobposting...


In [ ]:
# Convert the CSV to Excel format
excel_filename = "jobbank_jobs.xlsx"
df.to_excel(excel_filename, index=False)
print(f"Excel file saved as {excel_filename}")


Excel file saved as jobbank_jobs.xlsx


In [ ]:
from google.colab import files
files.download("jobbank_jobs.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def parse_posting(url):
    print("Parsing:", url)
    r = requests.get(url, headers=headers, timeout=20)
    if r.status_code != 200:
        return {"URL": url, "Status": r.status_code}

    soup = BeautifulSoup(r.text, "html.parser")

    # Job title
    title_tag = soup.select_one("span.noc-title, h1")
    title = title_tag.get_text(strip=True) if title_tag else ""

    # Employer + City fallback from <meta description>
    meta_desc = soup.find("meta", {"name": "description"})
    employer, city, province = "", "", ""
    if meta_desc and meta_desc.get("content"):
        desc = meta_desc["content"]
        # Try extracting Employer
        m_emp = re.search(r"for a .* at ([A-Za-z0-9&'’\-\s]+?) At", desc)
        if m_emp:
            employer = m_emp.group(1).strip()
        # Try extracting City/Province
        m_loc = re.search(r"At ([A-Za-z\s]+),?\s?([A-Z]{2})?", desc)
        if m_loc:
            city = m_loc.group(1).strip()
            province = m_loc.group(2) if m_loc.group(2) else ""

    # Date Posted
    date_tag = soup.select_one('span[property="datePosted"]')
    date_posted = date_tag.get_text(strip=True).replace("Posted on", "").strip() if date_tag else ""

    # NOC Code
    noc_tag = soup.select_one("span.noc-no")
    noc_match = re.search(r"\d{4,5}", noc_tag.get_text()) if noc_tag else None
    noc = noc_match.group(0) if noc_match else ""

    # Wage
    wage_el = soup.find(string=re.compile("Wage|Salary", re.I))
    wage = wage_el.parent.get_text(strip=True) if wage_el else ""

    # Employment Type
    emp_type = extract_employment_type(soup)

    # Work Arrangement
    work_arr = extract_work_arrangement(soup)

    # Source Site & URL
    site, link = extract_source_site_and_url(soup)

    return {
        "Title": title,
        "Employer": employer,
        "City": city,
        "Province": province,
        "Wage": wage,
        "DatePosted": date_posted,
        "NOC": noc,
        "EmploymentType": emp_type,
        "WorkArrangement": work_arr,
        "SourceSite": site,
        "SourceURL": link,
        "URL": url
    }


In [2]:
from google.colab import files
uploaded = files.upload()


Saving jobbank_jobs.xlsx to jobbank_jobs.xlsx


In [3]:
import pandas as pd
list(uploaded.keys())


['jobbank_jobs.xlsx']

In [6]:
# --- PHASE 2.1: DATA CLEANING & PREPROCESSING ---

import pandas as pd
import numpy as np
import re

# Load dataset (Excel version)
df = pd.read_excel("jobbank_jobs.xlsx", engine="openpyxl")

# --- General cleaning ---
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
df.replace(["", "None", None, np.nan], "N/A", inplace=True)
df.drop_duplicates(subset=["url"], inplace=True)

# --- Column-specific cleaning ---

# Title
if "title" in df.columns:
    df["title"] = df["title"].astype(str).str.title().str.replace(r"\(.*?\)", "", regex=True).str.strip()

# Employer
if "employer" in df.columns:
    df["employer"] = df["employer"].astype(str).str.replace(r"Employer\s*Details.*", "", regex=True)
    df["employer"] = df["employer"].str.replace(r"Employer\s*", "", regex=True).str.strip()

# City / Province / Postal code
def normalize_province(p):
    prov_map = {
        "Alberta": "AB", "British Columbia": "BC", "Manitoba": "MB", "New Brunswick": "NB",
        "Newfoundland and Labrador": "NL", "Nova Scotia": "NS", "Northwest Territories": "NT",
        "Nunavut": "NU", "Ontario": "ON", "Prince Edward Island": "PE", "Quebec": "QC",
        "Saskatchewan": "SK", "Yukon": "YT"
    }
    if isinstance(p, str):
        p = p.strip().title()
        return prov_map.get(p, p if p in prov_map.values() else "N/A")
    return "N/A"

if "province" in df.columns:
    df["province"] = df["province"].apply(normalize_province)

postal_pattern = re.compile(r"[A-Z]\d[A-Z]\s?\d[A-Z]\d")
if "postal_code" in df.columns:
    df["postal_code"] = df["postal_code"].apply(
        lambda x: x if isinstance(x, str) and postal_pattern.match(x) else "N/A"
    )

# Wage
if "wage" in df.columns:
    df["wage_amount"] = df["wage"].astype(str).str.extract(r"(\$?\d+(?:\.\d{1,2})?)")[0]
    df["wage_unit"] = df["wage"].astype(str).str.extract(r"(hour|week|month|year)", flags=re.I)[0]
    df["wage_amount"] = (
        df["wage_amount"].replace(r"[^0-9.]", "", regex=True).replace("", "N/A")
    )

# Date
if "date_posted" in df.columns:
    df["date_posted"] = pd.to_datetime(df["date_posted"], errors="coerce")
    df["date_posted"] = df["date_posted"].dt.strftime("%Y-%m-%d").fillna("N/A")

# NOC
if "noc" in df.columns:
    df["noc"] = df["noc"].apply(
        lambda x: x if re.match(r"^\d{4,5}$", str(x)) else "N/A"
    )

# Employment type
valid_types = ["Full-Time", "Part-Time", "Contract", "Temporary", "Casual", "Seasonal"]
if "employment_type" not in df.columns:
    df["employment_type"] = "N/A"
else:
    df["employment_type"] = df["employment_type"].astype(str).str.title()
df.loc[~df["employment_type"].isin(valid_types), "employment_type"] = "N/A"

# Work arrangement
valid_work = ["On-Site", "Remote/Hybrid"]
if "work_arrangement" not in df.columns:
    df["work_arrangement"] = "N/A"
else:
    df["work_arrangement"] = df["work_arrangement"].astype(str).str.title()
df.loc[~df["work_arrangement"].isin(valid_work), "work_arrangement"] = "N/A"

# Source site & URL
if "source_site" not in df.columns:
    df["source_site"] = "N/A"
else:
    df["source_site"] = df["source_site"].astype(str).str.extract(r"([A-Za-z0-9.-]+)")

if "source_url" not in df.columns:
    df["source_url"] = "N/A"
else:
    df["source_url"] = df["source_url"].apply(
        lambda x: x if isinstance(x, str) and x.startswith("http") else "N/A"
    )

# --- Validation filters ---
df = df[
    (df["title"] != "N/A")
    & (df["employer"] != "N/A")
    & (df["url"] != "N/A")
]

# --- Export cleaned dataset ---
df.to_csv("jobbank_cleaned.csv", index=False, encoding="utf-8")

# Log rows containing N/A values
cleaning_log = df[df.isin(["N/A"]).any(axis=1)]
cleaning_log.to_csv("jobbank_cleaning_log.csv", index=False, encoding="utf-8")

print("✅ Cleaned dataset saved as 'jobbank_cleaned.csv'")
print("🧾 Cleaning log saved as 'jobbank_cleaning_log.csv'")
print(f"📊 Final dataset shape: {df.shape}")


✅ Cleaned dataset saved as 'jobbank_cleaned.csv'
🧾 Cleaning log saved as 'jobbank_cleaning_log.csv'
📊 Final dataset shape: (16, 19)


/tmp/ipython-input-2492196550.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["wage_amount"].replace(r"[^0-9.]", "", regex=True).replace("", "N/A")


In [17]:
# --- PHASE 2.2: FEATURE ENGINEERING & LABEL ENCODING (Updated Safe Version) ---
import pandas as pd
import numpy as np
import re
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Load cleaned dataset
df = pd.read_csv("jobbank_cleaned.csv")

# ==================== 1️⃣  DERIVED FEATURES ====================

# --- Job Seniority ---
def get_seniority(title):
    title = str(title).lower()
    if any(k in title for k in ["senior", "lead", "principal", "manager", "head"]):
        return "Senior"
    elif any(k in title for k in ["junior", "entry", "assistant", "associate"]):
        return "Junior"
    else:
        return "Mid"

df["job_seniority"] = df["title"].apply(get_seniority)

# --- Wage Normalization (to hourly) ---
def wage_to_hourly(amount, unit, raw_wage):
    try:
        # First, try to use extracted amount and unit
        if pd.notna(amount) and pd.notna(unit):
            amount = float(re.sub(r'[$,]', '', str(amount)))
            unit = str(unit).lower()
            if "hour" in unit:
                return amount
            elif "week" in unit:
                return amount / 40          # assume 40 hours per week
            elif "month" in unit:
                return amount / (4 * 40)    # 4 weeks per month (approx)
            elif "year" in unit:
                return amount / (52 * 40)   # 52 weeks per year
        # If extraction failed, try to parse from raw_wage string
        if isinstance(raw_wage, str):
            raw_wage = raw_wage.lower()
            # Look for hourly rate
            hourly_match = re.search(r'(\$?\d+(\.\d{1,2})?)\s*per\s*hour', raw_wage)
            if hourly_match:
                return float(re.sub(r'[$,]', '', hourly_match.group(1)))
            # Look for weekly rate
            weekly_match = re.search(r'(\$?\d+(\.\d{1,2})?)\s*per\s*week', raw_wage)
            if weekly_match:
                 return float(re.sub(r'[$,]', '', weekly_match.group(1))) / 40
            # Look for monthly rate
            monthly_match = re.search(r'(\$?\d+(\.\d{1,2})?)\s*per\s*month', raw_wage)
            if monthly_match:
                 return float(re.sub(r'[$,]', '', monthly_match.group(1))) / (4 * 40)
             # Look for yearly rate
            yearly_match = re.search(r'(\$?\d+(\.\d{1,2})?)\s*per\s*year', raw_wage)
            if yearly_match:
                 return float(re.sub(r'[$,]', '', yearly_match.group(1))) / (52 * 40)

    except Exception as e:
        print(f"Error parsing wage '{raw_wage}': {e}") # Optional: log errors

    return np.nan

df["wage_hourly"] = df.apply(lambda x: wage_to_hourly(x.get("wage_amount"), x.get("wage_unit"), x.get("wage")), axis=1)


# --- Days Since Posted (auto-detect column) ---
date_col = None
for c in df.columns:
    if "date" in c.lower() and "post" in c.lower():
        date_col = c
        break

if date_col and date_col in df.columns:
    df["days_since_posted"] = (
        datetime.now() - pd.to_datetime(df[date_col], errors="coerce")
    ).dt.days
    # Fill NaN with median only if there are non-NaN values
    if df["days_since_posted"].notna().any():
        df["days_since_posted"] = df["days_since_posted"].fillna(df["days_since_posted"].median())
    else:
        df["days_since_posted"] = 0 # Or another default if all are NaN
else:
    df["days_since_posted"] = 0 # Default if date column not found


# --- Province Code ---
province_map = {
    "AB": 1, "BC": 2, "MB": 3, "NB": 4, "NL": 5, "NS": 6,
    "NT": 7, "NU": 8, "ON": 9, "PE": 10, "QC": 11, "SK": 12, "YT": 13
}
# Use the cleaned 'province' column from the previous step
if "province" in df.columns:
    df["province_code"] = df["province"].map(province_map).fillna(0).astype(int)
else:
     df["province_code"] = 0 # Default if province column not found


# ==================== 2️⃣  LABEL ENCODING ====================
enc_cols = ["employment_type", "work_arrangement", "province", "source_site"]
label_maps = {}

for col in enc_cols:
    # Check if the column exists and has non-NaN values before encoding
    if col in df.columns and df[col].notna().any():
        le = LabelEncoder()
        # Convert to string to handle potential mixed types and NaNs safely for LabelEncoder
        df[col + "_encoded"] = le.fit_transform(df[col].astype(str))
        label_maps[col] = dict(zip(le.classes_, le.transform(le.classes_)))
    else:
        # Create encoded column with default value if original column is missing or all NaN
        df[col + "_encoded"] = 0
        label_maps[col] = {} # Empty map if no values to encode


# ==================== 3️⃣  NORMALIZATION ====================
scaler = MinMaxScaler()
num_cols_to_scale = ["wage_hourly", "days_since_posted"]

# Select only the numeric columns that exist in the dataframe
existing_num_cols = [col for col in num_cols_to_scale if col in df.columns and pd.api.types.is_numeric_dtype(df[col])]

if existing_num_cols:
    # Fill NaNs with median before scaling, but only if there are non-NaN values
    for col in existing_num_cols:
        if df[col].notna().any():
            df[col] = df[col].fillna(df[col].median())
        else:
            df[col] = 0 # Or another appropriate default if all are NaN

    df[existing_num_cols] = scaler.fit_transform(df[existing_num_cols])
else:
    print("Warning: No numeric columns found for scaling.")


# ==================== 4️⃣  OUTPUT FILES ====================
df.to_csv("jobbank_features.csv", index=False)

with open("feature_summary.txt", "w", encoding="utf-8") as f:
    f.write("=== Feature Engineering Summary ===\n\n")
    f.write(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}\n\n")
    f.write("Encoded Label Maps:\n")
    for col, mapping in label_maps.items():
        f.write(f"\n{col}:\n{mapping}\n")

print("✅ Feature-engineered dataset saved as 'jobbank_features.csv'")
print("🧾 Summary file saved as 'feature_summary.txt'")
print(f"📊 Final dataset shape: {df.shape}")

✅ Feature-engineered dataset saved as 'jobbank_features.csv'
🧾 Summary file saved as 'feature_summary.txt'
📊 Final dataset shape: (16, 27)


In [10]:
import pandas as pd

df = pd.read_csv("jobbank_features.csv")
print(df.shape)
df.head()


(16, 27)


,title,employer,city,province,postalcode,wage,dateposted,noc,employmenttype,workarrangement,...,source_site,source_url,job_seniority,wage_hourly,days_since_posted,province_code,employment_type_encoded,work_arrangement_encoded,province_encoded,source_site_encoded
0,Big Data Analyst,Auto List,Winnipeg Region,NaN,NaN,Labour market informationExplore the marketJob...,"October 03, 2025",21211,Full Time,Remote/Hybrid,...,NaN,NaN,Mid,NaN,0.214286,0,0,0,0,0
1,Big Data Analyst,Big Way Hot Pot,Lower Mainland–Southwest Region,NaN,NaN,Labour market informationExplore the marketJob...,"October 18, 2025",21211,Full Time,Remote/Hybrid,...,NaN,NaN,Mid,NaN,0.035714,0,0,0,0,0
2,Data Mining Analyst,EXFO inc,Capitale-Nationale Region,NaN,NaN,Labour market informationExplore the marketJob...,"September 30, 2025",21211,NaN,Remote/Hybrid,...,NaN,NaN,Mid,NaN,0.250000,0,0,0,0,0
3,Data Mining Analyst Student Job,Farm Credit Canada,Regina–Moose Mountain Region,NaN,NaN,Labour market informationExplore the marketJob...,"September 24, 2025",21211,Full Time,Remote/Hybrid,...,NaN,NaN,Mid,NaN,0.321429,0,0,0,0,0
4,"Analyst, Database",Maxim Truck and Trailer,Interlake Region,NaN,NaN,Labour market informationExplore the marketJob...,"October 16, 2025",21223,Full Time,Remote/Hybrid,...,NaN,NaN,Mid,NaN,0.059524,0,0,0,0,0


In [11]:
feature_cols = [
    "wage_hourly", "days_since_posted", "province_code",
    "employment_type_encoded", "work_arrangement_encoded"
]
X = df[feature_cols]


In [12]:
y = df["wage_hourly"]


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import joblib
import numpy as np
import pandas as pd

# Load the feature-engineered data
df = pd.read_csv("jobbank_features.csv")

# Drop rows with missing target (wage_hourly)
df = df.dropna(subset=["wage_hourly"])

# Check if there are still rows left after dropping NaNs
if df.shape[0] == 0:
    print("Error: No data remaining after dropping rows with missing 'wage_hourly'. Cannot train models.")
else:
    # Define feature columns (excluding the target variable itself)
    feature_cols = ["days_since_posted", "province_code",
                    "employment_type_encoded", "work_arrangement_encoded"]

    # Ensure all feature columns exist before selecting
    existing_feature_cols = [col for col in feature_cols if col in df.columns]

    if not existing_feature_cols:
         print("Error: None of the specified feature columns exist in the dataframe.")
    else:
        X = df[existing_feature_cols]
        y = df["wage_hourly"]

        # It's generally better to handle missing values in features before splitting.
        # For simplicity with this small dataset, we'll assume features are handled
        # in the cleaning/feature engineering step. If there were NaNs in features
        # after dropping rows with missing target, you would need to impute or drop here.
        # Given the previous output, features seem to be filled (though perhaps with 0 if all were NaN).


        # Split
        # Adjust test_size if the number of samples is very small
        test_size = 0.2 if df.shape[0] >= 5 else (0.5 if df.shape[0] >= 2 else 0) # Ensure at least 1 sample in train and test if possible

        if test_size > 0 and df.shape[0] > 1:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42
            )

            # Check again if train/test sets are not empty after splitting
            if X_train.shape[0] == 0 or X_test.shape[0] == 0:
                 print(f"Error: Train or test set is empty after splitting. Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}. Cannot train models.")
            else:
                # Train models
                dt = DecisionTreeRegressor(random_state=42)
                rf = RandomForestRegressor(n_estimators=200, random_state=42)

                dt.fit(X_train, y_train)
                rf.fit(X_train, y_train) # Corrected y_test to y_train

                # Evaluate
                pred_dt = dt.predict(X_test)
                pred_rf = rf.predict(X_test)

                print("Decision Tree R²:", round(r2_score(y_test, pred_dt), 3))
                print("Random Forest R²:", round(r2_score(y_test, pred_rf), 3))

                # Save models
                joblib.dump(dt, "model_decisiontree.pkl")
                joblib.dump(rf, "model_randomforest.pkl")

                print("✅ Models trained and saved successfully.")
        else:
            print(f"Not enough data ({df.shape[0]} samples) after dropping missing wages to perform train/test split and train models.")

Decision Tree R²: 1.0
Random Forest R²: 1.0
✅ Models trained and saved successfully.


In [21]:
import joblib
joblib.dump(dt, "model_decisiontree.pkl")
joblib.dump(rf, "model_randomforest.pkl")


['model_randomforest.pkl']

In [22]:
!pip install PyPDF2 python-docx docx2txt spacy pandas
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!apt install tesseract-ocr -y
!pip install pytesseract pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [5]:
import re
import spacy
import pandas as pd
import docx2txt
from PyPDF2 import PdfReader
from google.colab import files

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_file(uploaded_file):
    """Extract text from .pdf or .docx"""
    if uploaded_file.name.endswith(".pdf"):
        pdf = PdfReader(uploaded_file)
        return " ".join(page.extract_text() for page in pdf.pages if page.extract_text())
    elif uploaded_file.name.endswith(".docx"):
        return docx2txt.process(uploaded_file)
    else:
        raise ValueError("Unsupported file format. Please upload .pdf or .docx")

def extract_entities(text):
    """Extract key entities from resume text"""
    doc = nlp(text)
    name = next((ent.text for ent in doc.ents if ent.label_ == "PERSON"), "N/A")
    email = re.search(r'[\w\.-]+@[\w\.-]+', text)
    phone = re.search(r'\+?\d[\d -]{8,12}\d', text)
    skills = [token.text for token in doc if token.pos_ == "NOUN" and len(token.text) > 2]
    return {
        "Name": name,
        "Email": email.group(0) if email else "N/A",
        "Phone": phone.group(0) if phone else "N/A",
        "Skills": ", ".join(set(skills))
    }

# Upload a resume
uploaded = files.upload()
file_name = next(iter(uploaded))
with open(file_name, "rb") as f:
    resume_text = extract_text_from_file(f)

# Parse resume content
parsed_data = extract_entities(resume_text)

# Convert to DataFrame and export
df = pd.DataFrame([parsed_data])
df.to_csv("parsed_resume.csv", index=False)
print("✅ Resume parsed successfully → saved as 'parsed_resume.csv'")
df


Saving Resume Sample PDF.pdf to Resume Sample PDF (3).pdf
✅ Resume parsed successfully → saved as 'parsed_resume.csv'


,Name,Email,Phone,Skills
0,N/A,N/A,N/A,


In [1]:
!pip install PyPDF2 docx2txt spacy pandas
!python -m spacy download en_core_web_sm

import re, spacy, docx2txt
import pandas as pd
from PyPDF2 import PdfReader
from google.colab import files

nlp = spacy.load("en_core_web_sm")

def extract_text(file):
    if file.name.endswith(".pdf"):
        reader = PdfReader(file)
        return " ".join([p.extract_text() or "" for p in reader.pages])
    elif file.name.endswith(".docx"):
        return docx2txt.process(file)
    else:
        raise ValueError("Unsupported format")

def parse_resume(text):
    doc = nlp(text)
    data = {}

    # --- Basic Info ---
    data["Name"] = next((ent.text for ent in doc.ents if ent.label_ == "PERSON"), "N/A")
    data["Email"] = re.search(r'[\w\.-]+@[\w\.-]+', text)
    data["Email"] = data["Email"].group(0) if data["Email"] else "N/A"
    data["Phone"] = re.search(r'\+?\d[\d\s\-\(\)]{8,}\d', text)
    data["Phone"] = data["Phone"].group(0) if data["Phone"] else "N/A"

    # --- Education ---
    edu_keywords = ["B.Sc", "M.Sc", "Bachelor", "Master", "Diploma", "PhD", "B.Tech", "MBA"]
    edu_lines = [line for line in text.split("\n") if any(k in line for k in edu_keywords)]
    data["Education"] = "; ".join(set(edu_lines)) if edu_lines else "N/A"

    # --- Experience ---
    exp_keywords = ["experience", "worked", "responsible", "project", "role"]
    exp_lines = [line for line in text.split("\n") if any(k in line.lower() for k in exp_keywords)]
    data["Experience"] = " ".join(exp_lines[:3]) if exp_lines else "N/A"

    # --- Skills ---
    skills_list = ["python", "excel", "sql", "tableau", "power bi", "machine learning",
                   "data analysis", "communication", "leadership", "statistics"]
    skills_found = [s for s in skills_list if re.search(rf'\b{s}\b', text.lower())]
    data["Skills"] = ", ".join(set(skills_found)) if skills_found else "N/A"

    # --- Certifications ---
    cert_keywords = ["certificate", "certified", "certification"]
    cert_lines = [line for line in text.split("\n") if any(k in line.lower() for k in cert_keywords)]
    data["Certifications"] = "; ".join(cert_lines) if cert_lines else "N/A"

    return data

# Upload and parse
uploaded = files.upload()
fname = next(iter(uploaded))
with open(fname, "rb") as f:
    text = extract_text(f)
parsed = parse_resume(text)

# Save
df = pd.DataFrame([parsed])
df.to_csv("parsed_resume_detailed.csv", index=False)
print("✅ Enhanced parsing complete → saved as 'parsed_resume_detailed.csv'")
df


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Saving Resume Sample PDF.pdf to Resume Sample PDF (5).pdf
✅ Enhanced parsing complete → saved as 'parsed_resume_detailed.csv'


,Name,Email,Phone,Education,Experience,Skills,Certifications
0,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [4]:
!apt install poppler-utils -y
!pip install pdf2image


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.11 [186 kB]
Fetched 186 kB in 0s (841 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126718 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...


In [10]:
!apt install tesseract-ocr -y
!pip install pytesseract pillow PyPDF2 pandas pdf2image

import pytesseract
from PIL import Image
from PyPDF2 import PdfReader
import pandas as pd
from pdf2image import convert_from_path
from google.colab import files
import re

uploaded = files.upload()
fname = next(iter(uploaded))

# Convert PDF pages to images
pages = convert_from_path(fname, 300)

# OCR extract
text = ""
for page in pages:
    text += pytesseract.image_to_string(page)

# --- Name Extraction (Enhanced) ---
lines = [l.strip() for l in text.splitlines() if l.strip()]
name = "N/A"
for line in lines[:10]:  # examine first 10 lines for potential name
    if not re.search(r'@|\d|objective|summary|experience|education', line, re.I):
        name = line
        break

# --- Email Extraction ---
email = re.search(r'[\w\.-]+@[\w\.-]+', text)

# --- Phone Extraction ---
phone = re.search(r'\+?\d[\d\s\-\(\)]{8,}\d', text)

# --- Skills Extraction ---
skills = []
keywords = ["Python", "Excel", "SQL", "Tableau", "Power BI", "Machine Learning",
            "Communication", "Leadership", "Data Analysis", "Statistics",
            "Java", "C++", "R", "AWS", "TensorFlow"]
for k in keywords:
    if re.search(rf'\b{k}\b', text, re.I):
        skills.append(k)

# --- Education Extraction ---
education = "N/A"
edu_match = re.search(r"(Bachelor|Master|B\.Sc|M\.Sc|B\.Eng|MBA|Ph\.D)[^,\n]*", text, re.I)
if edu_match:
    education = edu_match.group(0)

# --- Experience Extraction (Enhanced) ---
experience = "N/A"

# Numeric years (e.g., "3 years", "5+ yrs")
exp_match = re.search(r"(\d+)\+?\s*(year|yr)s?\b", text, re.I)
if exp_match:
    experience = exp_match.group(0)

# Range-based years (e.g., "2018–2022" or "2019 - Present")
elif re.search(r"(19|20)\d{2}\s*[-–]\s*((19|20)\d{2}|Present|Current)", text, re.I):
    experience = "Experience range mentioned"

# Descriptive (e.g., "over five years", "more than 10 years")
elif re.search(r"(over|more than|approximately)\s+\d+\s*(year|yr)s?", text, re.I):
    experience = re.search(r"(over|more than|approximately)\s+\d+\s*(year|yr)s?", text, re.I).group(0)

# --- Save Data ---
data = {
    "Name": name.strip() if name else "N/A",
    "Email": email.group(0) if email else "N/A",
    "Phone": phone.group(0) if phone else "N/A",
    "Education": education,
    "Experience": experience,
    "Skills": ", ".join(skills) if skills else "N/A"
}

df = pd.DataFrame([data])
df.to_csv("parsed_resume_ocr_detailed.csv", index=False)
print("✅ OCR Resume Parsing (Enhanced) Complete → saved as 'parsed_resume_ocr_detailed.csv'")
df


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


Saving Data Analyst - ats compatible-1.pdf to Data Analyst - ats compatible-1 (2).pdf
✅ OCR Resume Parsing (Enhanced) Complete → saved as 'parsed_resume_ocr_detailed.csv'


,Name,Email,Phone,Education,Experience,Skills
0,JOHN DOE,professionalemail@resumeworded.com,+1-234-456-789,Bachelor of Business Management,10 years,"Python, Tableau, Data Analysis"
